# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 25 2022 12:14PM,238072,18,6502716,Hush Hush,Released
1,May 25 2022 12:07PM,238068,19,GP05262022,"Granules Pharmaceuticals, Inc.",Released
2,May 25 2022 12:06PM,238071,10,MET210578,"Methapharm, Inc.",Released
3,May 25 2022 12:05PM,238067,10,MET210561,"Methapharm, Inc.",Released
4,May 25 2022 12:05PM,238067,10,MET210563,"Methapharm, Inc.",Released
5,May 25 2022 12:05PM,238067,10,MET210566,"Methapharm, Inc.",Released
6,May 25 2022 12:05PM,238067,10,MET210568,"Methapharm, Inc.",Released
7,May 25 2022 12:05PM,238067,10,MET210570,"Methapharm, Inc.",Released
8,May 25 2022 12:05PM,238067,10,MET210571,"Methapharm, Inc.",Released
9,May 25 2022 12:05PM,238067,10,MET210572,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,238068,Released,1
32,238069,Released,6
33,238070,Released,1
34,238071,Released,1
35,238072,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238068,NaN,NaN,1.0
238069,NaN,NaN,6.0
238070,NaN,NaN,1.0
238071,NaN,NaN,1.0
238072,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237975,14.0,2.0,0.0
237980,0.0,0.0,1.0
237981,0.0,0.0,1.0
237989,20.0,28.0,0.0
238013,0.0,43.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237975,14,2,0
237980,0,0,1
237981,0,0,1
237989,20,28,0
238013,0,43,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237975,14,2,0
1,237980,0,0,1
2,237981,0,0,1
3,237989,20,28,0
4,238013,0,43,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237975,14,2,
1,237980,,,1
2,237981,,,1
3,237989,20,28,
4,238013,,43,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 25 2022 12:14PM,238072,18,Hush Hush
1,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc."
2,May 25 2022 12:06PM,238071,10,"Methapharm, Inc."
3,May 25 2022 12:05PM,238067,10,"Methapharm, Inc."
28,May 25 2022 12:04PM,238069,10,"Methapharm, Inc."
34,May 25 2022 12:04PM,238070,10,"Methapharm, Inc."
35,May 25 2022 12:02PM,238066,10,"Methapharm, Inc."
57,May 25 2022 10:55AM,238059,10,"Methapharm, Inc."
62,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.
65,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 25 2022 12:14PM,238072,18,Hush Hush,,,1
1,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc.",,,1
2,May 25 2022 12:06PM,238071,10,"Methapharm, Inc.",,,1
3,May 25 2022 12:05PM,238067,10,"Methapharm, Inc.",,,25
4,May 25 2022 12:04PM,238069,10,"Methapharm, Inc.",,,6
5,May 25 2022 12:04PM,238070,10,"Methapharm, Inc.",,,1
6,May 25 2022 12:02PM,238066,10,"Methapharm, Inc.",,,22
7,May 25 2022 10:55AM,238059,10,"Methapharm, Inc.",,,5
8,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.,,,3
9,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 12:14PM,238072,18,Hush Hush,1,,
1,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc.",1,,
2,May 25 2022 12:06PM,238071,10,"Methapharm, Inc.",1,,
3,May 25 2022 12:05PM,238067,10,"Methapharm, Inc.",25,,
4,May 25 2022 12:04PM,238069,10,"Methapharm, Inc.",6,,
5,May 25 2022 12:04PM,238070,10,"Methapharm, Inc.",1,,
6,May 25 2022 12:02PM,238066,10,"Methapharm, Inc.",22,,
7,May 25 2022 10:55AM,238059,10,"Methapharm, Inc.",5,,
8,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.,3,,
9,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 12:14PM,238072,18,Hush Hush,1,,
1,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc.",1,,
2,May 25 2022 12:06PM,238071,10,"Methapharm, Inc.",1,,
3,May 25 2022 12:05PM,238067,10,"Methapharm, Inc.",25,,
4,May 25 2022 12:04PM,238069,10,"Methapharm, Inc.",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 12:14PM,238072,18,Hush Hush,1.0,NaN,NaN
1,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
2,May 25 2022 12:06PM,238071,10,"Methapharm, Inc.",1.0,NaN,NaN
3,May 25 2022 12:05PM,238067,10,"Methapharm, Inc.",25.0,NaN,NaN
4,May 25 2022 12:04PM,238069,10,"Methapharm, Inc.",6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 12:14PM,238072,18,Hush Hush,1.0,0.0,0.0
1,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
2,May 25 2022 12:06PM,238071,10,"Methapharm, Inc.",1.0,0.0,0.0
3,May 25 2022 12:05PM,238067,10,"Methapharm, Inc.",25.0,0.0,0.0
4,May 25 2022 12:04PM,238069,10,"Methapharm, Inc.",6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2618607,89.0,65.0,2.0
15,714123,16.0,32.0,0.0
16,238025,1.0,0.0,0.0
18,952114,4.0,19.0,0.0
19,952141,3.0,2.0,14.0
20,476010,25.0,38.0,20.0
21,237980,1.0,0.0,0.0
22,238050,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2618607,89.0,65.0,2.0
1,15,714123,16.0,32.0,0.0
2,16,238025,1.0,0.0,0.0
3,18,952114,4.0,19.0,0.0
4,19,952141,3.0,2.0,14.0
5,20,476010,25.0,38.0,20.0
6,21,237980,1.0,0.0,0.0
7,22,238050,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,89.0,65.0,2.0
1,15,16.0,32.0,0.0
2,16,1.0,0.0,0.0
3,18,4.0,19.0,0.0
4,19,3.0,2.0,14.0
5,20,25.0,38.0,20.0
6,21,1.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,89.0
1,15,Released,16.0
2,16,Released,1.0
3,18,Released,4.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,2.0,0.0,0.0,0.0,14.0,20.0,0.0,0.0
Executing,65.0,32.0,0.0,19.0,2.0,38.0,0.0,0.0
Released,89.0,16.0,1.0,4.0,3.0,25.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,14.0,20.0,0.0,0.0
1,Executing,65.0,32.0,0.0,19.0,2.0,38.0,0.0,0.0
2,Released,89.0,16.0,1.0,4.0,3.0,25.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,14.0,20.0,0.0,0.0
1,Executing,65.0,32.0,0.0,19.0,2.0,38.0,0.0,0.0
2,Released,89.0,16.0,1.0,4.0,3.0,25.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()